In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from xgboost import XGBRegressor
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_parquet("../data/training/water_consumption_training.parquet")
df.head()

,average_input_flow_rate_24_hours,average_change_reservoir_level_percentage_24_hours,average_total_liters_entered_24_hours,sum_total_liters_entered_last_24_hours,average_effective_liters_entered_24_hours,sum_effective_liters_entered_last_24_hours,average_total_liters_out_last_24_hours,sum_total_liters_out_last_24_hours,average_output_flow_rate_last_24_hours,average_pressure_last_24_hours,average_total_precip_mm_last_24_hours,average_station_pressure_mb_last_24_hours,average_max_pressure_last_hour_mb_last_24_hours,average_min_pressure_last_hour_mb_last_24_hours,average_global_radiation_kj_m2_last_24_hours,average_air_temp_c_last_24_hours,average_dew_point_temp_c_last_24_hours,average_max_temp_last_hour_c_last_24_hours,average_min_temp_last_hour_c_last_24_hours,average_max_dew_point_last_hour_c_last_24_hours,average_min_dew_point_last_hour_c_last_24_hours,average_max_humidity_last_hour_percentage_last_24_hours,average_min_humidity_last_hour_percentage_last_24_hours,average_relative_humidity_percentage_last_24_hours,average_wind_direction_deg_last_24_hours,average_max_wind_gust_m_s_last_24_hours,average_wind_speed_m_s_last_24_hours,average_input_flow_rate_10_hours,average_change_reservoir_level_percentage_10_hours,average_total_liters_entered_10_hours,sum_total_liters_entered_last_10_hours,average_effective_liters_entered_10_hours,sum_effective_liters_entered_last_10_hours,average_total_liters_out_last_10_hours,sum_total_liters_out_last_10_hours,average_output_flow_rate_last_10_hours,average_pressure_last_10_hours,average_total_precip_mm_last_10_hours,average_station_pressure_mb_last_10_hours,average_max_pressure_last_hour_mb_last_10_hours,average_min_pressure_last_hour_mb_last_10_hours,average_global_radiation_kj_m2_last_10_hours,average_air_temp_c_last_10_hours,average_dew_point_temp_c_last_10_hours,average_max_temp_last_hour_c_last_10_hours,average_min_temp_last_hour_c_last_10_hours,average_max_dew_point_last_hour_c_last_10_hours,average_min_dew_point_last_hour_c_last_10_hours,average_max_humidity_last_hour_percentage_last_10_hours,average_min_humidity_last_hour_percentage_last_10_hours,average_relative_humidity_percentage_last_10_hours,average_wind_direction_deg_last_10_hours,average_max_wind_gust_m_s_last_10_hours,average_wind_speed_m_s_last_10_hours,average_input_flow_rate_1_hour,average_change_reservoir_level_percentage_1_hour,average_total_liters_entered_1_hour,sum_total_liters_entered_last_1_hour,average_effective_liters_entered_1_hour,sum_effective_liters_entered_last_1_hour,average_total_liters_out_last_1_hour,sum_total_liters_out_last_1_hour,average_output_flow_rate_last_1_hour,average_pressure_last_1_hour,average_total_precip_mm_last_1_hour,average_station_pressure_mb_last_1_hour,average_max_pressure_last_hour_mb_last_1_hour,average_min_pressure_last_hour_mb_last_1_hour,average_global_radiation_kj_m2_last_1_hour,average_air_temp_c_last_1_hour,average_dew_point_temp_c_last_1_hour,average_max_temp_last_hour_c_last_1_hour,average_min_temp_last_hour_c_last_1_hour,average_max_dew_point_last_hour_c_last_1_hour,average_min_dew_point_last_hour_c_last_1_hour,average_max_humidity_last_hour_percentage_last_1_hour,average_min_humidity_last_hour_percentage_last_1_hour,average_relative_humidity_percentage_last_1_hour,average_wind_direction_deg_last_1_hour,average_max_wind_gust_m_s_last_1_hour,average_wind_speed_m_s_last_1_hour,average_input_flow_rate_10_minutes,average_change_reservoir_level_percentage_10_minutes,average_total_liters_entered_10_minutes,sum_total_liters_entered_last_10_minutes,average_effective_liters_entered_10_minutes,sum_effective_liters_entered_last_10_minutes,average_total_liters_out_last_10_minutes,sum_total_liters_out_last_10_minutes,average_output_flow_rate_last_10_minutes,average_pressure_last_10_minutes,average_total_precip_mm_last_10_minutes,average_station_pressure_mb_last_10_minutes,average_max_pressure_last_hour_mb_last_10_minutes,average_min_pressure_last_hour_mb_last_10_minutes,average_global_radiat

In [4]:
weather_columns = [
    'total_precip_mm', 
    'station_pressure_mb', 
    'max_pressure_last_hour_mb',
    'min_pressure_last_hour_mb', 
    'global_radiation_kj_m2', 
    'air_temp_c',
    'dew_point_temp_c', 
    'max_temp_last_hour_c', 
    'min_temp_last_hour_c',
    'max_dew_point_last_hour_c', 
    'min_dew_point_last_hour_c',
    'max_humidity_last_hour_percentage', 
    'min_humidity_last_hour_percentage',
    'relative_humidity_percentage', 
    'wind_direction_deg', 
    'max_wind_gust_m_s',
    'wind_speed_m_s'
]

weather_feature_columns = []

for window_name in ['24_hours', '10_hours', '1_hour', '10_minutes']:
    weather_feature_columns.extend([f'average_{col}_last_{window_name}' for col in weather_columns])
    weather_feature_columns.extend([f'last_{col}' for col in weather_columns])

# Include timestamp in the list
columns_to_exclude = ['timestamp'] + weather_feature_columns
training_df = df.drop(columns=columns_to_exclude)
training_df.head()

,average_input_flow_rate_24_hours,average_change_reservoir_level_percentage_24_hours,average_total_liters_entered_24_hours,sum_total_liters_entered_last_24_hours,average_effective_liters_entered_24_hours,sum_effective_liters_entered_last_24_hours,average_total_liters_out_last_24_hours,sum_total_liters_out_last_24_hours,average_output_flow_rate_last_24_hours,average_pressure_last_24_hours,average_input_flow_rate_10_hours,average_change_reservoir_level_percentage_10_hours,average_total_liters_entered_10_hours,sum_total_liters_entered_last_10_hours,average_effective_liters_entered_10_hours,sum_effective_liters_entered_last_10_hours,average_total_liters_out_last_10_hours,sum_total_liters_out_last_10_hours,average_output_flow_rate_last_10_hours,average_pressure_last_10_hours,average_input_flow_rate_1_hour,average_change_reservoir_level_percentage_1_hour,average_total_liters_entered_1_hour,sum_total_liters_entered_last_1_hour,average_effective_liters_entered_1_hour,sum_effective_liters_entered_last_1_hour,average_total_liters_out_last_1_hour,sum_total_liters_out_last_1_hour,average_output_flow_rate_last_1_hour,average_pressure_last_1_hour,average_input_flow_rate_10_minutes,average_change_reservoir_level_percentage_10_minutes,average_total_liters_entered_10_minutes,sum_total_liters_entered_last_10_minutes,average_effective_liters_entered_10_minutes,sum_effective_liters_entered_last_10_minutes,average_total_liters_out_last_10_minutes,sum_total_liters_out_last_10_minutes,average_output_flow_rate_last_10_minutes,average_pressure_last_10_minutes,last_input_flow_rate,last_reservoir_level_percentage,last_total_liters_entered,last_effective_liters_entered,last_total_liters_out,last_output_flow_rate,last_pressure,total_time_pump_1_was_on_last_24_hours,total_time_pump_2_was_on_last_24_hours,last_pump_1_status,last_pump_2_status,second,minute,hour,day,weekday,week_of_year,month,year,output_flow_rate
0,44.306319,45.528108,8309.346181,4786183.4,2611.111111,1504000.0,8280.873958,4769783.4,43.716719,36.048941,49.168667,56.468500,9942.8540,2386284.96,2915.000000,699600.0,10022.020667,2405284.96,46.705542,37.371500,10.924167,46.328333,2451.687500,58840.50,625.000000,15000.0,8601.687500,206440.50,45.727083,38.173750,0.0,39.040,0.0,0.0,0.0,0.0,7500.0,30000.0,40.8350,38.415,0.0,37.50,0.0,0.0,11000.0,36.67,38.63,0.0,61800.0,0,0,53,0,19,18,5,11,3,2023,36.67
1,44.192361,45.528003,8292.252431,4776337.4,2605.902778,1501000.0,8293.294097,4776937.4,43.799514,36.049479,48.888250,56.449083,9858.7290,2366094.96,2858.333333,686000.0,10052.895667,2412694.96,47.002875,37.373333,8.222083,45.624167,1643.764583,39450.35,316.666667,7600.0,8685.431250,208450.35,47.945000,38.182083,0.0,37.940,0.0,0.0,0.0,0.0,11000.0,44000.0,64.1675,38.425,0.0,36.10,0.0,0.0,14000.0,93.33,38.37,0.0,61650.0,0,0,23,3,19,18,5,11,3,2023,93.33
2,44.078403,45.527309,8275.158681,4766491.4,2600.000000,1497600.0,8282.103125,4770491.4,43.724913,36.049896,48.607833,56.426750,9816.6665,2355999.96,2829.166667,679000.0,10039.999833,2409599.96,46.916917,37.375417,5.520000,44.920000,1235.750000,29658.00,316.666667,7600.0,8277.416667,198658.00,45.242917,38.187917,0.0,37.075,0.0,0.0,0.0,0.0,8650.0,34600.0,48.5000,38.425,0.0,36.10,0.0,0.0,0.0,0.00,38.27,0.0,61500.0,0,0,53,5,19,18,5,11,3,2023,0.00
3,43.964444,45.524462,8258.064931,4756645.4,2594.791667,1494600.0,8286.537153,4773045.4,43.754479,36.050243,48.334333,56.398417,9775.6415,2346153.96,2808.333333,674000.0,10058.974833,2414153.96,47.043417,37.377375,2.801667,44.145000,422.968333,10151.24,0.000000,0.0,8172.968333,196151.24,46.195000,38.202083,0.0,36.215,0.0,0.0,0.0,0.0,8600.0,34400.0,48.1675,38.410,0.0,35.16,0.0,0.0,9400.0,62.67,38.37,0.0,61350.0,0,0,23,8,19,18,5,11,3,2023,62.67
4,43.851181,45.519948,8241.075347,4746859.4,2594.791667,1494600.0,8286.214236,4772859.4,43.752326,36.050590,48.060833,56.366083,9734.6165,2336307.96,2808.333333,674000.0,10057.949833,2413907.96,47.036583,37.379333,0.083333,43.330000,12.500000,300.00,0.000000,0.0,8162.500000,195

In [5]:
# Assuming 'timestamp' is the column with datetime information
train = training_df[training_df['year'] == 2023]
test = training_df[training_df['year'] == 2024]

# List of features for training (excluding 'output_flow_rate')
training_features = [col for col in training_df.columns if col != 'output_flow_rate']

# Create X and y
X_train = train[training_features]
y_train = train['output_flow_rate']
X_test = test[training_features]
y_test = test['output_flow_rate']

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(92837, 59) (92837,)
(31659, 59) (31659,)


In [6]:
def set_model_training_pipeline() -> GridSearchCV:
    # Initialize the XGBoost regressor with specified hyperparameters
    model = XGBRegressor(
        n_estimators=100000, 
        learning_rate=0.01, 
        early_stopping_rounds=100
    )
    
    # Define time series cross-validation strategy
    cv = TimeSeriesSplit(n_splits=5)
    
    # Define an expanded parameter grid for hyperparameter tuning
    params = {
        'n_estimators': [100, 500, 1000, 5000],
        'max_depth': [3, 5, 10, 14],
        'learning_rate': [0.01, 0.05, 0.1]
    }
    
    # Initialize GridSearchCV with the model, parameter grid, and cross-validation strategy
    clf = GridSearchCV(
        estimator=model, 
        param_grid=params, 
        cv=cv, 
        scoring='neg_mean_squared_error',
        n_jobs=-1,  # Use all available cores
        verbose=2
    )
    
    return clf

def train_model(clf: GridSearchCV, x_train: pd.DataFrame, y_train: pd.Series, x_test: pd.DataFrame, y_test: pd.Series) -> GridSearchCV:
    # Train the model with training and validation data
    clf.fit(
        x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        verbose=100
    )
    
    return clf.best_estimator_
    
    
def calculate_error(y_test: pd.Series, y_pred: pd.Series):
    # Calculate error metrics
    mae = round(mean_absolute_error(y_true=y_test, y_pred=y_pred), 2)
    mse = round(mean_squared_error(y_true=y_test, y_pred=y_pred), 2)
    rmse = round(np.sqrt(mse), 2)
    r2 = round(r2_score(y_true=y_test, y_pred=y_pred), 2)
    
    return mae, mse, rmse, r2

def predict_with_model(clf: GridSearchCV, X: pd.DataFrame) -> pd.Series:
    # Generate predictions using the trained model
    y_pred = clf.predict(X)
    
    return pd.Series(y_pred)

In [7]:
clf = set_model_training_pipeline()
clf = train_model(clf, X_train, y_train, X_test, y_test)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[0]	validation_0-rmse:30.40405	validation_1-rmse:31.02632
[0]	validation_0-rmse:29.72174	validation_1-rmse:29.70650
[0]	validation_0-rmse:30.40405	validation_1-rmse:31.02632
[0]	validation_0-rmse:29.47081	validation_1-rmse:29.03022
[0]	validation_0-rmse:29.72174	validation_1-rmse:29.70650
[0]	validation_0-rmse:29.47081	validation_1-rmse:29.03022
[0]	validation_0-rmse:29.37729	validation_1-rmse:28.70510
[0]	validation_0-rmse:29.34540	validation_1-rmse:28.45743
[99]	validation_0-rmse:17.88566	validation_1-rmse:15.02334
[100]	validation_0-rmse:17.94962	validation_1-rmse:15.55199
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   7.9s
[99]	validation_0-rmse:18.01244	validation_1-rmse:15.63228
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=   8.0s
[99]	validation_0-rmse:16.82988	validation_1-rmse:14.57069
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=100; total time=  

In [10]:
mae, mse, rmse, r2 = calculate_error(y_test, predict_with_model(clf, X_test))
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

MAE: 0.3
MSE: 48.1
RMSE: 6.94
R2: 0.94


In [11]:
import pickle
model_path = "../models/xgb_flow_out_forecast_3.pkl"
pickle.dump(clf, open(model_path, 'wb'))